### Importing the libraries

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

### Preparing and loading the dataset from DKube

In [33]:
DATA_DIR = '/opt/dkube/input'
mnist = np.load(DATA_DIR+'/mnist.npz')

In [34]:
def get_dataset(flag):
    x_train=mnist['x_train']
    x_test=mnist['x_test']
    y_train=mnist['y_train']
    y_test=mnist['y_test']
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train[..., tf.newaxis].astype("float32")
    x_test = x_test[..., tf.newaxis].astype("float32")
    if flag=='training':
        return x_train,y_train
    if flag=='testing':
        return x_test,y_test

In [43]:
def train_dataset():
    x_train,y_train=get_dataset('training')
    return (tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32))

In [44]:
def test_dataset():
    x_test,y_test=get_dataset('testing')
    return(tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32))

### Building the Model

In [45]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

### Choose an optimizer and loss function for training

In [46]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

### Select the metrics

In [47]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Use tf.GradientTape to train the model

In [48]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

### Test the model

In [49]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [50]:
EPOCHS = 5

train_ds=train_dataset()
test_ds=test_dataset()

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.1341385543346405, Accuracy: 95.8933334350586, Test Loss: 0.058173950761556625, Test Accuracy: 98.18000030517578
Epoch 2, Loss: 0.043357767164707184, Accuracy: 98.63666534423828, Test Loss: 0.054844871163368225, Test Accuracy: 98.1500015258789
Epoch 3, Loss: 0.022055676206946373, Accuracy: 99.34333801269531, Test Loss: 0.06457231938838959, Test Accuracy: 98.19999694824219
Epoch 4, Loss: 0.014255895279347897, Accuracy: 99.54499816894531, Test Loss: 0.058675412088632584, Test Accuracy: 98.43000030517578
Epoch 5, Loss: 0.009897562675178051, Accuracy: 99.64833068847656, Test Loss: 0.0683283656835556, Test Accuracy: 98.15999603271484
